# Deploy ADK to Agent Engine


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.sandbox.google.com/github/hupili/google-adk-101/blob/main/Deploy_to_Agent_Engine.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fhupili%2Fgoogle-adk-101%2Fmain%2FDeploy_to_Agent_Engine.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-enterprise-logo.png" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/hupili/google-adk-101/blob/main/Deploy_to_Agent_Engine.ipynb">
      <img width="32px" src="https://www.gstatic.com/monospace/240802/git_host_github_mask.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/hupili/google-adk-101/blob/main/Deploy_to_Agent_Engine.ipynb">
      <img width="32px" src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

| | |
|-|-|
| Author(s) | [Pili Hu](https://github.com/hupili)|
| Reviewers(s) | Nil |
| Last updated | 2025-06-02 |
| google-adk | 1.1.1 |
| vertexai | 1.93.1 |

## Environment

In [1]:
import os

# API_KEY = '' # @param {type:"string"}
# os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE" # Use Vertex AI API
# os.environ["GOOGLE_API_KEY"] = API_KEY

PROJECT_ID = "hupili-genai-bb"  # @param {type:"string"}
if not PROJECT_ID:
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "us-central1" # @param {type:"string"}

STAGING_BUCKET = 'gs://agent_engine_deploy_staging' # @param {type:"string"}

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE" # Use Vertex AI API

# [your-project-id]

In [2]:
import vertexai

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

In [3]:
from google.colab import auth
auth.authenticate_user()

In [4]:
!pip install google-adk==1.1.1

In [5]:
from google import adk

In [6]:
adk.__version__

'1.1.1'

In [7]:
vertexai.__version__

'1.93.1'

## AE 101

In [8]:
from vertexai import agent_engines

In [9]:
def delete(resource_id: str) -> None:
    remote_agent = agent_engines.get(resource_id)
    remote_agent.delete(force=True)
    print(f"Deleted remote agent: {resource_id}")

def list_agents() -> None:
    remote_agents = agent_engines.list()
    TEMPLATE = '''
{agent.name} ("{agent.display_name}")
- Create time: {agent.create_time}
- Update time: {agent.update_time}
'''
    remote_agents_string = '\n'.join(TEMPLATE.format(agent=agent) for agent in remote_agents)
    print(f"All remote agents:\n{remote_agents_string}")

In [10]:
list_agents()

All remote agents:
 
2164164338904989696 ("")
- Create time: 2025-06-02 11:57:47.050939+00:00
- Update time: 2025-06-02 12:01:18.059991+00:00

 
6087608854902734848 ("DICE_AGENT")
- Create time: 2025-04-25 08:19:20.817490+00:00
- Update time: 2025-04-25 08:22:05.936293+00:00

 
7916070303615156224 ("llm_auditor")
- Create time: 2025-04-25 07:13:25.339138+00:00
- Update time: 2025-04-25 07:18:12.898631+00:00

 
5307078744484085760 ("llm_auditor")
- Create time: 2025-04-09 08:22:36.179879+00:00
- Update time: 2025-04-09 08:25:42.734584+00:00

 
8402318325882814464 ("")
- Create time: 2025-03-28 08:42:23.121480+00:00
- Update time: 2025-03-28 08:46:29.629436+00:00

 
66410502317670400 ("my-awesome-agent")
- Create time: 2025-03-07 01:22:05.930759+00:00
- Update time: 2025-03-07 01:33:02.201959+00:00



## ADK Agent

In [11]:
from google.adk.tools import google_search
from google.adk import Agent
from google.adk.agents import Agent, LlmAgent
from google.genai import types
from pydantic import BaseModel
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

MODEL = "gemini-2.0-flash"
APP_NAME = "simple_search_example"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "simple_search_agent"

# Agent
simple_search_agent = Agent(
    model=MODEL,
    name="simple_search_agent",
    description="Agent to answer questions using Google Search.",
    instruction="I can answer your questions by searching the internet. Just ask me anything!",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[google_search],
)

# Session and Runner
session_service = InMemorySessionService()
session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=simple_search_agent, app_name=APP_NAME, session_service=session_service)

# Agent Interaction
def call_agent(runner, session, query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  events = runner.run(user_id=session.user_id, session_id=session.id, new_message=content)
  return events

events = call_agent(runner, session, "what is next Google IO date?")
list(events)

[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='Google I/O 2025 is scheduled for May 20-21, 2025. It will be held at the Shoreline Amphitheatre in Mountain View, California, and will also be available online.\n')], role='model'), grounding_metadata=GroundingMetadata(grounding_chunks=[GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(domain='blog.google', title='blog.google', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEHK_fhUfIWG7YgLgiXDPypopgWjomUQG2hm5oTzLBl8cq2ysysxzJvFhrhAE3Tt1G2EyCoPm-pBOgUb5BNApZF-p9Ta8OepopBy96l3yFeJM_nbPUxULQKNeZUPQXq-tZ8VdLl8RIIDVte5eJD0oH2-w==')), GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(domain='indianexpress.com', title='indianexpress.com', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXECuwHBmIT4H2QgcZw2AVvjexSwr_wKyA

In [12]:
#

## Agent Engine Local

In [13]:
from vertexai.preview import reasoning_engines

app = reasoning_engines.AdkApp(
    agent=simple_search_agent,
    enable_tracing=True,
)

In [14]:
session = await app.create_session(user_id="u_123")
session

Session(id='a02ddb99-2a27-4e18-b2e7-c21e0257e67d', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1748873651.3863306)

In [15]:
await app.list_sessions(user_id="u_123")

ListSessionsResponse(sessions=[Session(id='a02ddb99-2a27-4e18-b2e7-c21e0257e67d', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1748873651.3863306)])

In [16]:
my_session = await app.get_session(user_id="u_123", session_id=session.id)

In [17]:
my_session

Session(id='a02ddb99-2a27-4e18-b2e7-c21e0257e67d', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1748873651.3863306)

In [18]:
for event in app.stream_query(
    user_id="u_123",
    session_id=my_session.id,
    message="whats the weather in new york",
):
  print(event)

{'content': {'parts': [{'text': 'The weather in New York is currently sunny with a temperature of 65°F (18°C). It feels like 67°F (20°C) with 44% humidity. The forecast for today is partly cloudy during the day and clear at night, with a 10% chance of rain. The temperature will be between 56°F (13°C) and 72°F (22°C).\n'}], 'role': 'model'}, 'grounding_metadata': {'grounding_chunks': [{'web': {'domain': 'google.com', 'title': 'Weather information for locality: New York, administrative_area: NY', 'uri': 'https://www.google.com/search?q=weather+in+New York,+NY'}}], 'grounding_supports': [{'confidence_scores': [0.71043974], 'grounding_chunk_indices': [0], 'segment': {'end_index': 79, 'text': 'The weather in New York is currently sunny with a temperature of 65°F (18°C).'}}, {'confidence_scores': [0.92162645], 'grounding_chunk_indices': [0], 'segment': {'end_index': 126, 'start_index': 80, 'text': 'It feels like 67°F (20°C) with 44% humidity.'}}, {'confidence_scores': [0.88870764], 'groundin

## Agent Engine Remote

In [19]:
from vertexai import agent_engines

In [21]:
remote_app = agent_engines.create(
    agent_engine=simple_search_agent,
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]"
    ]
)

In [37]:
remote_app.resource_name

'projects/160214671203/locations/us-central1/reasoningEngines/2164164338904989696'

In [22]:
# Get the previously created AE App
# remote_app = agent_engines.get('projects/160214671203/locations/us-central1/reasoningEngines/2164164338904989696')

In [23]:
remote_session = remote_app.create_session(user_id="u_456")
remote_session

{'appName': '2164164338904989696',
 'events': [],
 'state': {},
 'id': '7853974834680692736',
 'userId': 'u_456',
 'lastUpdateTime': 1748873730.686434}

In [24]:
remote_app.list_sessions(user_id="u_456")

{'sessions': [{'appName': '2164164338904989696',
   'events': [],
   'state': {},
   'id': '7853974834680692736',
   'userId': 'u_456',
   'lastUpdateTime': 1748873730.686434},
  {'appName': '2164164338904989696',
   'events': [],
   'state': {},
   'id': '6737082127092809728',
   'lastUpdateTime': 1748872990.284667,
   'userId': 'u_456'},
  {'appName': '2164164338904989696',
   'events': [],
   'state': {},
   'id': '6088563780751458304',
   'userId': 'u_456',
   'lastUpdateTime': 1748872979.004046},
  {'appName': '2164164338904989696',
   'events': [],
   'state': {},
   'id': '2125396108665421824',
   'lastUpdateTime': 1748870518.234342,
   'userId': 'u_456'}]}

In [25]:
remote_app

resource name: projects/160214671203/locations/us-central1/reasoningEngines/2164164338904989696

In [26]:
for event in remote_app.stream_query(
    user_id="u_456",
    session_id=remote_session["id"],
    message="whats the weather in Hong Kong",
):
    print(event)

{'content': {'parts': [{'text': "The weather in Hong Kong is currently cloudy with a temperature of 84°F (29°C), but it feels like 94°F (34°C) due to the humidity of around 81%. There is a 15% chance of rain.\n\nHere's the forecast for the next few days:\n\n*   **Tuesday, June 3:** Light rain, with temperatures between 79°F (26°C) and 84°F (29°C).\n*   **Wednesday, June 4:** Light rain, with temperatures between 77°F (25°C) and 81°F (27°C).\n*   **Thursday, June 5:** Light rain, with temperatures between 79°F (26°C) and 82°F (28°C).\n"}], 'role': 'model'}, 'grounding_metadata': {'grounding_chunks': [{'web': {'domain': 'google.com', 'title': 'Weather information for query: weather in Hong Kong', 'uri': 'https://www.google.com/search?q=weather+in+Hong+Kong'}}], 'grounding_supports': [{'confidence_scores': [0.90916675], 'grounding_chunk_indices': [0], 'segment': {'end_index': 148, 'text': 'The weather in Hong Kong is currently cloudy with a temperature of 84°F (29°C), but it feels like 94

In [27]:
for event in remote_app.stream_query(
    user_id="u_456",
    session_id=remote_session["id"],
    message="Remember my home location is Hong Kong.",
):
    print(event)

{'content': {'parts': [{'text': 'OK. I will remember that your home location is Hong Kong.\n'}], 'role': 'model'}, 'grounding_metadata': {}, 'usage_metadata': {'candidates_token_count': 14, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 14}], 'prompt_token_count': 250, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 250}], 'total_token_count': 264, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-7e7f8c16-c21e-44cf-914c-80e32459c85d', 'author': 'simple_search_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'MAwpGYuk', 'timestamp': 1748873792.683036}


In [28]:
new_remote_app = agent_engines.get(remote_app.resource_name)

In [29]:
remote_session2 = remote_app.create_session(user_id="u_456")
remote_session2

{'appName': '2164164338904989696',
 'events': [],
 'state': {},
 'id': '1009347826007670784',
 'lastUpdateTime': 1748873801.712665,
 'userId': 'u_456'}

In [30]:
for event in remote_app.stream_query(
    user_id="u_456",
    session_id=remote_session["id"],
    message="what is my home location?",
):
    print(event)

{'content': {'parts': [{'text': 'Your home location is Hong Kong.\n'}], 'role': 'model'}, 'grounding_metadata': {}, 'usage_metadata': {'candidates_token_count': 8, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 8}], 'prompt_token_count': 270, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 270}], 'total_token_count': 278, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-3e09a1bb-0481-4a03-9510-7cd1b73ae171', 'author': 'simple_search_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': '9UGGbm8C', 'timestamp': 1748873803.378211}


In [31]:
for event in remote_app.stream_query(
    user_id="u_456",
    session_id=remote_session2["id"],
    message="what is my home location?",
):
    print(event)

{'content': {'parts': [{'text': 'I do not have access to your personal information, including your home location. I am a large language model; therefore, I cannot ask you for your Personally Identifiable Information (PII). That includes your home address.\n'}], 'role': 'model'}, 'grounding_metadata': {}, 'usage_metadata': {'candidates_token_count': 44, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 44}], 'prompt_token_count': 53, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 53}], 'total_token_count': 97, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-c1010fc6-9130-4e49-9c24-4f21bf874a6f', 'author': 'simple_search_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'BfPSnRnv', 'timestamp': 1748873805.861248}


In [32]:
def call_remote_agent_engine_app(app, user_id, session_id, query):
  response_text = ''
  for event in app.stream_query(
      user_id=user_id,
      session_id=session_id,
      message=query,
  ):
      print(event)
      all_parts = event.get('content', {}).get('parts', [])
      for part in all_parts:
          response_text += (part.get('text', ''))
  return response_text

In [33]:
call_remote_agent_engine_app(remote_app, "u_456", remote_session2["id"], "what is my home location?")

{'content': {'parts': [{'text': 'As a large language model, I have no memory of past conversations and I do not have access to your personal information. Therefore, I do not know your home location. I cannot ask you for your Personally Identifiable Information (PII).\n'}], 'role': 'model'}, 'grounding_metadata': {}, 'usage_metadata': {'candidates_token_count': 48, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 48}], 'prompt_token_count': 103, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 103}], 'total_token_count': 151, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-33eca1b8-9310-4343-8597-2bd2fb2100cb', 'author': 'simple_search_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'YemtjLHQ', 'timestamp': 1748873812.385044}


'As a large language model, I have no memory of past conversations and I do not have access to your personal information. Therefore, I do not know your home location. I cannot ask you for your Personally Identifiable Information (PII).\n'

In [34]:
call_remote_agent_engine_app(remote_app, "u_456", remote_session["id"], "what is my home location?")

{'content': {'parts': [{'text': 'Your home location is Hong Kong.\n'}], 'role': 'model'}, 'grounding_metadata': {'retrieval_metadata': {}, 'search_entry_point': {'rendered_content': '<style>\n.container {\n  align-items: center;\n  border-radius: 8px;\n  display: flex;\n  font-family: Google Sans, Roboto, sans-serif;\n  font-size: 14px;\n  line-height: 20px;\n  padding: 8px 12px;\n}\n.chip {\n  display: inline-block;\n  border: solid 1px;\n  border-radius: 16px;\n  min-width: 14px;\n  padding: 5px 16px;\n  text-align: center;\n  user-select: none;\n  margin: 0 8px;\n  -webkit-tap-highlight-color: transparent;\n}\n.carousel {\n  overflow: auto;\n  scrollbar-width: none;\n  white-space: nowrap;\n  margin-right: -12px;\n}\n.headline {\n  display: flex;\n  margin-right: 4px;\n}\n.gradient-container {\n  position: relative;\n}\n.gradient {\n  position: absolute;\n  transform: translate(3px, -9px);\n  height: 36px;\n  width: 9px;\n}\n@media (prefers-color-scheme: light) {\n  .container {\n  

'Your home location is Hong Kong.\n'

## Mesop

In [35]:
!pip install mesop

In [36]:
import mesop as me
import mesop.labs as mel

me.colab_run()


Running server on: http://localhost:32123


In [37]:
@me.page(path="/hello_world")
def hello_world():
  me.text("Hello World!")

In [38]:
me.colab_show(path="/hello_world", height=100)

<IPython.core.display.Javascript object>

In [39]:
@me.page(path="/chat")
def chat():
  mel.chat(transform)

def transform(prompt: str, history: list[mel.ChatMessage]) -> str:
  return "Hello " + prompt

In [40]:
me.colab_show(path="/chat")

<IPython.core.display.Javascript object>

In [41]:
USER_ID = 'mesop_user_123'
SESSION = remote_app.create_session(user_id=USER_ID)

@me.page(path="/chat_ae")
def chat():
  mel.chat(transform)

def transform(prompt: str, history: list[mel.ChatMessage]) -> str:
  # No need to use history, as AE already manages the session.
  return call_remote_agent_engine_app(remote_app, USER_ID, SESSION['id'], prompt)

In [42]:
me.colab_show(path="/chat_ae")

<IPython.core.display.Javascript object>

### Screenshots

The dynamic web UI may not present on the export notebook. Below are the screenshots.

In [43]:
from IPython.display import Image, display

In [44]:
display(Image(url='https://github.com/hupili/google-adk-101/raw/main/images/ss-ae-mesop-1.png'))

In [45]:
display(Image(url='https://github.com/hupili/google-adk-101/raw/main/images/ss-ae-mesop-2.png'))